In [1]:
import pandas as pd

from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score

from sklearn import model_selection, metrics, preprocessing, neighbors

In [2]:
df_train = pd.read_csv('../../Feature_Encoding/data/train_helmert_encoding.csv')
df_test = pd.read_csv('../../Feature_Encoding/data/test_helmert_encoding.csv')
train = pd.read_csv('../../Feature_Engineering/data/other-cleaned_train.csv')

In [3]:
def cross_val(model, x_train, y_train):
    score_cross_val = model_selection.cross_val_score(model, x_train, y_train, cv=5)
    print(score_cross_val.mean())

In [4]:
def test_model(model, x_test, y_test):
    predictions = model.predict_proba(x_test)[:,1]
    logloss = metrics.log_loss(y_test, predictions)
    accuracy = metrics.accuracy_score(y_test, predictions.round())
    print("Accuracy: %.2f%%, Logloss: %.2f" % (accuracy*100.0, logloss))

In [5]:
#Realiza busqueda completa combinando los parametros
def knn(x_train, y_train, x_validation, y_validation):
    knn_classifier = neighbors.KNeighborsClassifier()
    params_knn = {'weights':['uniform','distance'],'leaf_size':[10,15,30,45],'n_neighbors':[5,10,25,50],'p':[1,2,3,4]}    
    knn_gs = model_selection.GridSearchCV(knn_classifier, params_knn, cv=2,verbose=3)
    knn_gs.fit(x_train, y_train)
    knn_best = knn_gs.best_estimator_
    print(knn_gs.best_params_)
    print('knn: {}'.format(knn_best.score(x_validation, y_validation)))
    return knn_best

In [6]:
y = train.Target
x_train, x_validation, y_train, y_validation = model_selection.train_test_split(df_train, y, test_size=0.3, stratify=y)

In [7]:
#Para normalizar las columnas
normalized_mean = preprocessing.normalize(df_train,axis=0)

y = train.Target
normalized_x_train, normalized_x_validation, normalized_y_train, normalized_y_validation_mean_encoding = model_selection.train_test_split(normalized_mean, y, test_size=0.3, stratify=y)

In [ ]:
knn_ = knn(x_train, y_train, x_validation, y_validation)
test_model(knn_, x_validation, y_validation)
cross_val(knn_, x_train, y_train)

Fitting 2 folds for each of 128 candidates, totalling 256 fits


In [ ]:
normalized_knn = knn(x_train, y_train, x_validation, y_validation)
test_model(normalized_knn, x_validation, y_validation)
cross_val(normalized_knn, x_train, y_train)

In [ ]:
for column in df_test.columns.tolist():
    df_test[column] = df_test[column].fillna(0)  

df_test= df_test.replace('', 0)

In [ ]:
y_pred_knn = knn_.predict_proba(df_test)[:,1]
submission_knn = pd.DataFrame(data={'Opportunity_ID':df_test['Opportunity_ID'], 'Target': y_pred_knn})
submission_knn = submission_knn.groupby("Opportunity_ID").agg({"Target":"mean"}).reset_index()
submission_knn.to_csv('../submits/helmert_knn.csv', index=False)

In [ ]:
y_pred_knn_norm = normalized_knn.predict_proba(df_test)[:,1]
submission_knn_norm = pd.DataFrame(data={'Opportunity_ID':df_test['Opportunity_ID'], 'Target': y_pred_knn_norm})
submission_knn_norm = submission_knn_norm.groupby("Opportunity_ID").agg({"Target":"mean"}).reset_index()
submission_knn_norm.to_csv('../submits/helmert_knn_norm.csv', index=False)